### Defining simple w3 functions when uniswapper object is not avaiable

In [ ]:
import os
os.environ['PROVIDER'] = 'https://goerli.prylabs.net/'

In [2]:
from web3 import Web3
from web3.middleware import geth_poa_middleware

class web3py:

    def __init__(self, endpoint=os.environ['PROVIDER']):
        self.w3 = Web3(Web3.HTTPProvider(endpoint))
        print('Connected: ', self.w3.isConnected())
        print('RPC Endpoint: ', endpoint)

        # If gorli testnet inject middleware
        if self.w3.net.version == '5' or w3.net.version == '4':
            self.w3.middleware_onion.inject(geth_poa_middleware, layer=0)
            print('POA middleware injected')

        # Get the number of the latest block
        last_block = self.w3.eth.getBlock('latest')
        print('Latest block: ', last_block['number'])

    def get_balance(self, address):
        return self.w3.eth.getBalance(address)

    def wei_to_eth(self, value):
        return self.w3.eth.web3.fromWei(value, 'ether')

    def get_tx_receipt(self, txhash):
        return self.w3.eth.getTransactionReceipt(txhash)

    def create_address(self):
        account = self.w3.eth.account.create()
        return (account.address, account.privateKey)

Consider installing rusty-rlp to improve pyrlp performance with a rust based backend


### Uniswapper functions

In [3]:
from uniswap import Uniswap

class UniSwapper:
    def __init__(self, address, private_key, entity_name, token_address=None, version=2):
        self.address = address
        self.private_key = private_key
        self.entity_name = entity_name
        self.version = version
        # Initialize uniswapper object
        self.uniswap_wrapper = Uniswap(address, private_key, version=version)  # pass version=2 to use Uniswap v2
        # Define contracts to interact with
        self.eth = "0x0000000000000000000000000000000000000000"
        self.weth = self.uniswap_wrapper.w3.toChecksumAddress("0xb4fbf271143f4fbf7b91a5ded31805e42b2208d6")
        # Set default KWH token address if not
        if token_address is None:
            self.token_address = self.uniswap_wrapper.w3.toChecksumAddress("0x78B57C213a18DF1DAbC647149902ea1966E0119C")
        else:
            self.token_address = self.uniswap_wrapper.w3.toChecksumAddress(token_address)
              
    def swap_eth_to_token(self, wei_amount, wait_tx_receipt=False):
        if wait_tx_receipt:
            tx = self.uniswap_wrapper.make_trade(self.token_address, self.eth, int(wei_amount))
            return self.uniswap_wrapper.w3.eth.waitForTransactionReceipt(tx)
        else:
            return self.uniswap_wrapper.make_trade(self.token_address, self.eth, int(wei_amount))

    def swap_token_to_eth(self, wei_amount, wait_tx_receipt=False):
        if wait_tx_receipt:
            tx = self.uniswap_wrapper.make_trade(self.eth, self.token_address, int(wei_amount))
            return self.uniswap_wrapper.w3.eth.waitForTransactionReceipt(tx)
        else:
            return self.uniswap_wrapper.make_trade(self.eth, self.token_address, int(wei_amount))

    def get_token_balance(self, convert_to_ether=False):
        if convert_to_ether:
            return self.uniswap_wrapper.get_token_balance(self.token_address)
        else:
            balance = self.uniswap_wrapper.get_token_balance(self.token_address)
            return self.uniswap_wrapper.w3.eth.web3.fromWei(balance, 'ether')
    
    def get_eth_balance(self, convert_to_ether=False):
        if convert_to_ether:
            return self.uniswap_wrapper.get_eth_balance()
        else:
            balance = self.uniswap_wrapper.get_eth_balance()
            return self.uniswap_wrapper.w3.eth.web3.fromWei(balance, 'ether')

In [4]:
w3 = web3py()

Connected:  True
RPC Endpoint:  https://goerli.prylabs.net/
POA middleware injected
Latest block:  3752313


In [5]:
account = w3.create_address()

In [6]:
u = UniSwapper(account[0], account[1], 'Community')

In [7]:
u.get_token_balance()

0

In [8]:
u.get_eth_balance()

0

In [9]:
u.entity_name

'Community'

In [10]:
address = "0xFe594E862c3ce76E192997EABFC41Afd7C975b52"
private_key = "3a8bb854c7a86d950c0d3e0b5b1bbcd3912389a95fa530e46c911fe1de099808"
account = (address, private_key)
u = UniSwapper(account[0], account[1], 'Adr')
amount = int(1/10000 *10**18)

In [11]:
u.get_token_balance(convert_to_ether=False)

Decimal('289268.613094465350825106')

In [12]:
u.get_token_balance(convert_to_ether=True)

289268613094465350825106

In [13]:
u.swap_eth_to_token(amount, wait_tx_receipt=True)

AttributeDict({'blockHash': HexBytes('0x2b8373e8f7fbec7cf0094c78cf124f91150a02e8ba3bf49b71705627417e7543'),
 'blockNumber': 3752315,
 'contractAddress': None,
 'cumulativeGasUsed': 708393,
 'from': '0xFe594E862c3ce76E192997EABFC41Afd7C975b52',
 'gasUsed': 125868,
 'logs': [AttributeDict({'address': '0x78B57C213a18DF1DAbC647149902ea1966E0119C',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x000000000000000000000000fe594e862c3ce76e192997eabfc41afd7c975b52'),
    HexBytes('0x0000000000000000000000002350a783efeb9322631f6b87fdebeb1852ad346d')],
   'data': '0x00000000000000000000000000000000000000000000000000005af3107a4000',
   'blockNumber': 3752315,
   'transactionHash': HexBytes('0x3fcea1420b44b70e42b5e577e382024e3c0011d4a4d45eff9bebe4cacf6c6960'),
   'transactionIndex': 5,
   'blockHash': HexBytes('0x2b8373e8f7fbec7cf0094c78cf124f91150a02e8ba3bf49b71705627417e7543'),
   'logIndex': 9,
   'removed': False}),
  AttributeDict({'